# 使用 LinearRNA 进行 RNA 二级结构预测

LinearRNA 包括一系列的线性时间 RNA 二级结构分析算法：**LinearFold** 和 **LinearPartition**。关于这个主题的更多信息请查阅[这里](https://github.com/PaddlePaddle/PaddleHelix/c/pahelix/toolkit/linear_rna/README_cn.md)。

## RNA 二级结构预测

**LinearFold** 是第一个以线性时间预测 RNA 二级结构的算法，可将 RNA 二级结构预测的时间大大降低。LinearFold 的论文已经在计算生物学顶级会议 ISMB 及生物信息学权威杂志 Bioinformatics 上发表。论文链接：[LinearFold: linear-time approximate RNA folding by 5'-to-3' dynamic programming and beam search](http://academic.oup.com/bioinformatics/article/35/14/i295/5529205)

# 第一部分：LinearFold
首先，我们需要安装pahelix依赖：

In [8]:
!pip install paddlehelix
from IPython.display import clear_output
clear_output()
print("安装成功")

安装成功


## 机器学习模型
```bash
linear_fold_c(rna_sequence, beam_size = 100, use_constraints = False, constraint = "", no_sharp_turn = True)
```
使用 [CONTRAfold](https://pubmed.ncbi.nlm.nih.gov/16873527/) 中的机器学习模型。

### 参数说明
* rna_sequence: string，需要预测结构的 RNA 序列。
* beam_size: int（默认值 100），控制 beam pruning size 的参数，设置为 0 关闭 beam pruning。该参数越大，则预测速度越慢，而与精确搜索相比近似效果越好。
* use_constraints: bool（默认值 False），在预测二级结构时增加约束条件。当为 True 时，constraint 参数需要提供约束序列。
* constraint: string（默认值空字符串），二级结构预测约束条件。当提供约束序列时，use_constraints 参数需要设置为 True。该约束须与输入的 RNA 序列长度相同，每个点位可以指定“? . ( )”四种符号中的一种，其中“?”表示该点位无限制，“.”表示该点位必须是 unpaired，“(”与“)”表示该点位必须是 paired。注意“(”与“)”必须数量相等，即相互匹配。具体操作请参考运行实例。
- no_sharp_turn: bool（默认值 True），不允许在预测的 hairpin 结构中出现 sharp turn。

### 返回值
* tuple(string, double): 返回一个二元组, 第一个位置是结构序列，第二个位置是结构的folding free energy。

In [9]:
import pahelix.toolkit.linear_rna as linear_rna
input_sequence = "AACUCCGCCAGGCCUGGAAGGGAGCAACGGUAGUGACACUCUCUGUGUGCGUAGGUUGCCUAGCUACCAUUU"
linear_rna.linear_fold_c(input_sequence)

('..((((.(((....)))...))))....((((((............................))))))....',
 0.4548597317188978)

In [10]:
# with constraints
constraint = "??(???(??????)?(????????)???(??????(???????)?)???????????)??.???????????"
linear_rna.linear_fold_c(input_sequence, use_constraints = True, constraint = constraint)

('..(.(((......)((........))(((......(.......).))).....))..)..............',
 -27.328358240425587)

## 热力学模型
```bash
linear_fold_v(rna_sequence, beam_size = 100, use_constraints = False, constraint = "", no_sharp_turn = True)
```
使用 [Vienna RNAfold](https://almob.biomedcentral.com/articles/10.1186/1748-7188-6-26) 中的热力学自由能模型。

参数和返回值与机器学习模型一致。

In [11]:
linear_rna.linear_fold_v(input_sequence)

('..((((.(((....)))...))))....((((((.((((.....))))...((((...))))))))))....',
 -18.4)

In [12]:
# with constriants
linear_rna.linear_fold_v(input_sequence, use_constraints = True, constraint = constraint)

('..(.(((......)((........))(((......(.......).))).....))..)..............',
 13.4)

# 第二部分：LinearPartition

**LinearPartition** 是世界上最快的 RNA 配分方程和碱基对概率预测算法。该算法功能更加强大，可以模拟 RNA 序列在平衡态时成千上万种不同结构的分布，并预测碱基对概率矩阵。LinearPartition 算法同样被 ISMB 顶会接收并在 Bioinformatics 杂志上发表。论文链接：[LinearPartition: linear-time approximation of RNA folding partition function and base-pairing probabilities](https://academic.oup.com/bioinformatics/article/36/Supplement_1/i258/5870487)

## 机器学习模型
```bash
linear_partition_c(rna_sequence, beam_size = 100, bp_cutoff = 0.0, no_sharpe_turn = True)
```
`linear_c`：使用 [CONTRAfold](https://pubmed.ncbi.nlm.nih.gov/16873527/) 中的机器学习模型。


### 参数说明
* rna_sequence: string，需要计算配分函数和碱基对概率的RNA序列。
* beam_size: int（默认值 100），控制 beam pruning size 的参数，默认值为 100。该参数越大，则预测速度越慢，而与精确搜索相比近似效果越好。
* bp_cutoff: double（默认值 0.9），只输出概率大于等于 bp_cutoff 的碱基对及其概率，0 <= bp_cutoff <= 1。
* no_sharp_turn: bool（默认值 True），不允许在预测的 hairpin 结构中出现 sharp turn。

### 返回值
* tuple(string, list): 返回一个二元组，第一个位置是配分函数值，第二个位置是存有碱基对及其概率的列表。

In [13]:
input_sequence = "UGAGUUCUCGAUCUCUAAAAUCG"
linear_rna.linear_partition_c("UGAGUUCUCGAUCUCUAAAAUCG", bp_cutoff = 0.2)

(0.6399469375610352,
 [(4, 13, 0.2007068395614624),
  (10, 22, 0.24661558866500854),
  (11, 21, 0.2457289695739746),
  (12, 20, 0.20926791429519653)])

### 热力学模型
```bash
linear_partition_v(rna_sequence, beam_size = 100, bp_cutoff = 0.0, no_sharpe_turn = True)
```
使用 [Vienna RNAfold](https://almob.biomedcentral.com/articles/10.1186/1748-7188-6-26) 中的热力学自由能模型。参数和返回值与机器学习模型一致。

In [14]:
linear_rna.linear_partition_v(input_sequence, bp_cutoff = 0.5)

(-1.9573111534118652,
 [(2, 15, 0.833134651184082),
  (3, 14, 0.8365526795387268),
  (4, 13, 0.8355389833450317)])